In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import argparse
import logging
from pathlib import Path

import numpy as np

In [ ]:
import sys
sys.path.append('../../source/')


In [ ]:

import HolographicLattices

In [ ]:
from HolographicLattices.DifferentialOperators.DifferentialOperator import \
    StructuredFiniteDifferenceDifferentialOperator
from HolographicLattices.Equations.EquationsOfMotion import EquationsOfMotion
from HolographicLattices.Equations.Jacobian import FiniteDifferenceJacobian
from HolographicLattices.IO.SeedLoader import load_seed_algebraic
from HolographicLattices.Physics.Observables import Observables
from HolographicLattices.LinearSolver.SimpleLinearSolver import SimpleLinearSolver
from HolographicLattices.NonLinearSolver.NRNonLinearSolver import NewtonRaphsonFiniteDifferenceBVPSolver as NRSolver
from HolographicLattices.Options import Options
from HolographicLattices.Utilities import GridUtilities

In [ ]:
    # Basic argparse for some settings. Can be extended, but not required.
    parser = argparse.ArgumentParser()

    parser.add_argument("--setupfile", type=Path, help="Setup .yaml file to read from")
    parser.add_argument("--output", type=Path, nargs="?", help="Where to output")
    parser.add_argument("--log", type=str, default="INFO",
                        help="Logging level to print, choice of [NONE|DEBUG|INFO|WARN|ERROR|CRITICAL]")
    parser.add_argument("--logfile", type=Path, default="SolveWithFD.log", help="Location of log file")
    parser.add_argument("--seedfile", type=Path, nargs="?")

    parsed, rest = parser.parse_known_args()

    logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", filename=parsed.logfile,
                        level=parsed.log)


In [ ]:
    # Load all the options
    #setupfile = parsed.setupfile
    
    setupfile="1DLattice_setup.yaml"

    io_options       = Options.IOOptions.load_from_file(setupfile)
    equation_options = Options.EquationOptions.load_from_file(setupfile)
    grid_options     = Options.GridOptions.load_from_file(setupfile)
    solver_options   = Options.NonLinearSolverOptions.load_from_file(setupfile)
    constant_options = Options.ConstantOptions.load_from_file_safe(setupfile)

    grid_dimensions = grid_options.grid_sizes